# Utilities for large scale climate data analysis
The `xarrayutils.utils` module contains several functions that have proven useful in several of my day to day projects with both observational and model data.

## Linear regression
One of the operations many scientists do is calculating a linear trend along a specified dimension (e.g. time) on each grid point of a dataset. `linear_trend` makes this very easy. For demonstration purposes lets load some monthly gridded Argo data from [APDRC](http://apdrc.soest.hawaii.edu/dods/public_data/Argo_Products/monthly_mean/monthly_mixed_layer.info)

In [1]:
import xarray as xr
import numpy as np

%matplotlib inline

In [2]:
path = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/Argo_Products/monthly_mean/monthly_mixed_layer'
ds = xr.open_dataset(path, use_cftime=True)
ds

<xarray.Dataset>
Dimensions:  (lat: 60, lon: 120, time: 240)
Coordinates:
  * time     (time) object 2001-01-15 00:00:00 ... 2020-12-15 00:00:00
  * lat      (lat) float64 -88.5 -85.5 -82.5 -79.5 -76.5 ... 79.5 82.5 85.5 88.5
  * lon      (lon) float64 1.5 4.5 7.5 10.5 13.5 ... 349.5 352.5 355.5 358.5
Data variables:
    mld      (time, lat, lon) float32 ...
    smld     (time, lat, lon) float32 ...
    nmld     (time, lat, lon) float32 ...
    ild      (time, lat, lon) float32 ...
    sild     (time, lat, lon) float32 ...
    nild     (time, lat, lon) float32 ...
    ttd      (time, lat, lon) float32 ...
    sttd     (time, lat, lon) float32 ...
    nttd     (time, lat, lon) float32 ...
    blt      (time, lat, lon) float32 ...
    sblt     (time, lat, lon) float32 ...
    nblt     (time, lat, lon) float32 ...
    tid      (time, lat, lon) float32 ...
    stid     (time, lat, lon) float32 ...
    ntid     (time, lat, lon) float32 ...
    mlt      (time, lat, lon) float32 ...
    smlt     (time, lat, lon) float32 ...
    nmlt     (time, lat, lon) float32 ...
    mls      (time, lat, lon) float32 ...
    smls     (time, lat, lon) float32 ...
    nmls     (time, lat, lon) float32 ...
Attributes:
    title:          3x3 bin-averaged Mixed Layer Monthly mean (from 2001)
    Conventions:    COARDS\nGrADS
    dataType:       Grid
    documentation:  http://apdrc.soest.hawaii.edu/projects/Argo/index.html
    history:        Wed Jan 06 09:40:44 HST 2021 : imported by GrADS Data Ser...

In [ ]:
ds.load()

Lets find out how much the salinity in each grid point changed over the full period (20 years)

In [3]:
from xarrayutils.utils import linear_trend

In [ ]:
# create an array 
salinity_regressed = linear_trend(ds.mls, 'time')
salinity_regressed

Now we can plot the slope as a map

In [ ]:
salinity_regressed.slope.plot(robust=True)

`linear_trend` converts the dimension over which to integrate into logical indicies, so the units of the plot above are (salinity/timestep of the original product), so here PSS/month.

## Correlation maps

But what about a bit more complex task? Lets find out how mixedlayer salinity and temperature correlate. For this we use `xr_linregress` (for which `linear_trend` is just a thin wrapper):

In [ ]:
from xarrayutils.utils import linear_trend, xr_linregress

In [ ]:
tempxsalt = xr_linregress(ds.mlt, ds.mls, dim='time')

In [ ]:
tempxsalt.r_value.plot()

This works in any dimension the dataset has:

In [ ]:
tempxsalt = xr_linregress(ds.mlt, ds.mls, dim='lon')

In [ ]:
tempxsalt.r_value.plot(x='time')

This map shows that in lower latitudes spatial patterns of salinity are generally anticorrlated with temperature and vice versa in the high latitudes.

## Hatch sign agreement

It can often be useful to indicate if the sign along an averaged (or otherwise aggregated) dimension. For instance to show if forced changes have a consistent sign in reference to a multi-member mean. `sign_agreement` makes this easy. Consider this synthetic example:

In [ ]:
x = np.linspace(-np.pi, np.pi, 25)
y = np.linspace(-np.pi, np.pi, 25)
xx, yy = np.meshgrid(x,y)

data1 = np.sin(xx)
data2 = np.sin(yy)
data3 = np.ones_like(xx)

np.fill_diagonal(data3,np.nan)
np.fill_diagonal(data3[1:],np.nan)
np.fill_diagonal(data3[:,1:],np.nan)
np.fill_diagonal(data3[:,2:],np.nan)

da = xr.DataArray(np.array([data1, data2, data3]), dims=['member','x', 'y'])

da.plot(col='member')

Taking the mean of these fields, suggests that values increase in the upper-left, upper-right, lower-right quadrant, and the missing values in the third layer distort the mean.

In [ ]:
da.mean('member').plot()

Lets produce a mask to see where all elements along the `member` dimension have the same sign:

In [ ]:
from xarrayutils.utils import sign_agreement

sign_agreement(da, da.mean('member'), 'member', threshold=1.0).plot()

You could use this information to indicate the areas of the average, where the members do not agree by hatching:

In [ ]:
da.mean('member').plot()
sign_agreement(
    da, da.mean('member'), 'member'
).plot.contourf(
    colors='none',
    hatches=['..', None],
    levels=[0,0.5],
    add_colorbar=False
)

## Masking values in the mixed layer

Sometimes it is helpful to analyze data by excluding the values in the mixed layer. This can be easily done with `mask_mixedlayer`. Let's see how:

First load a CMIP6 dataset from the cloud

In [ ]:
import intake
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
cat = col.search(
    table_id='Omon',
    grid_label='gn',
    experiment_id='historical',
    member_id='r1i1p1f1',
    variable_id=['thetao','mlotst'],#, 
    source_id=["ACCESS-ESM1-5"]
)
ddict = cat.to_dataset_dict(
    zarr_kwargs={'consolidated':True, 'decode_times':True},
)

In [ ]:
ds = ddict['CMIP.CSIRO.ACCESS-ESM1-5.historical.Omon.gn']
ds

We can remove the values in the mixed layer

In [ ]:
from xarrayutils.utils import mask_mixedlayer
ds_wo_ml = mask_mixedlayer(ds, ds.mlotst)
ds_wo_ml

Or to have the mixed layer values only

In [ ]:
from xarrayutils.utils import mask_mixedlayer
ds_ml_only = mask_mixedlayer(ds, ds.mlotst, mask='inside')
ds_ml_only

In [ ]:
import matplotlib.pyplot as plt
roi = dict(i=150, time=0, j=slice(50, 200))
plt.figure(figsize=[10, 15])
for di, (data, label) in enumerate(zip([ds, ds_wo_ml, ds_ml_only], ['full data', 'mixed layer removed', 'mixed layer only'])):
    plt.subplot(3,1,di+1)
    data.thetao.isel(lev=slice(0,20),**roi).plot(yincrease=False)
    ds.mlotst.isel(**roi).plot(x='j')
    plt.title(label)

In this case the cell bounds are not available for the model output, but `mask_mixedlayer` has the option to take those into account and e.g. remove cells only if the lower boundary is within the mixed layer. All you need to do is correctly specify `z_bounds` with the variable/coordinate name of the cell bounds.

## Removing bottom values

